<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [3]:
#ANSWER:
response.headers['Content-Type']

'application/json'

We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [4]:
#ANSWER:
parameters = {"lat": -33.87, "lon": 151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [5]:
#ANSWER:
response = requests.get("http://api.open-notify.org/iss-pass.json", params = parameters)

Print the response header:

In [6]:
#ANSWER:
print(response.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Sat, 19 Sep 2020 03:05:27 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}


Print the content of the response (the data that the server returned):

In [7]:
#ANSWER:
print(response.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1600484727, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 650, \n      "risetime": 1600489555\n    }, \n    {\n      "duration": 216, \n      "risetime": 1600495536\n    }, \n    {\n      "duration": 642, \n      "risetime": 1600543785\n    }, \n    {\n      "duration": 613, \n      "risetime": 1600549613\n    }, \n    {\n      "duration": 421, \n      "risetime": 1600555586\n    }\n  ]\n}\n'


Note that this is a Python byte string:

In [8]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [9]:
#ANSWER:
print(response.headers["content-type"])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [10]:
overheads = response.json()
print(overheads)

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1600484727, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 650, 'risetime': 1600489555}, {'duration': 216, 'risetime': 1600495536}, {'duration': 642, 'risetime': 1600543785}, {'duration': 613, 'risetime': 1600549613}, {'duration': 421, 'risetime': 1600555586}]}


What kind of object did this give us?

In [11]:
#ANSWER:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [12]:
#ANSWER:
passes = overheads['response']

In [13]:
passes[1]['risetime']

1600495536

In [14]:
x = []
for item in passes:
  x.append(item['risetime'])
x

[1600489555, 1600495536, 1600543785, 1600549613, 1600555586]

Now extract the `risetime` strings into an array called `srisetimes`:

In [15]:
#ANSWER:
srisetimes = [xpass['risetime'] for xpass in passes]
srisetimes

[1600489555, 1600495536, 1600543785, 1600549613, 1600555586]

```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

In [16]:
risetimes = [datetime.fromtimestamp(xpass['risetime']) for xpass in passes] 

Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [17]:
risetimes

[datetime.datetime(2020, 9, 19, 14, 25, 55),
 datetime.datetime(2020, 9, 19, 16, 5, 36),
 datetime.datetime(2020, 9, 20, 5, 29, 45),
 datetime.datetime(2020, 9, 20, 7, 6, 53),
 datetime.datetime(2020, 9, 20, 8, 46, 26)]

In [18]:
#ANSWER:
for risetime in risetimes:
  print(risetime.strftime('%d/%m/%y %I:%M'))

19/09/20 02:25
19/09/20 04:05
20/09/20 05:29
20/09/20 07:06
20/09/20 08:46


Here is an endpoint that tells us who is onboard:

In [19]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [20]:
#ANSWER:
astros = response.json()
print(astros)
print(astros["number"])
[print(astronaut['name']) for astronaut in astros['people']]

{'number': 3, 'people': [{'craft': 'ISS', 'name': 'Chris Cassidy'}, {'craft': 'ISS', 'name': 'Anatoly Ivanishin'}, {'craft': 'ISS', 'name': 'Ivan Vagner'}], 'message': 'success'}
3
Chris Cassidy
Anatoly Ivanishin
Ivan Vagner


[None, None, None]

## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [21]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    '''
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [22]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

TypeError: 'NoneType' object is not subscriptable

>

>

>



---



---



> > > > > > > > > © 2019 Institute of Data


---



---



